# Finetuning model

## Testing Fine-tuning

In [1]:
from huggingface_hub import notebook_login

notebook_login()

## Transforming dataset :

In [63]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

csv = "/content/drive/MyDrive/UH - Final Year Project/data/df-iter-1.csv"
df = pd.read_csv(csv)
df

,sentence,vars,names,variable_position
0,The loading term becomes relevant at a time τ,['τ'],a time,['end']
1,we get easily for the rupture force f ∗,['f'],the rupture force,['end']
2,Green’s theorem can be used to show that the v...,['d3xG(x)'],the volume,['end']
3,at a ﬁxed point x,['x'],['point'],['end']
4,it will be quite small (|K| is large) around t...,['x'],the point,['end']
...,...,...,...,...
296,we are inspired by the recent advances in the ...,['(information)'],quantum,['end']
297,"then the correlation function C(A, B)","['C(A, B)']",the correlation function,['end']
298,Next is to utilize the perturbation expansion ...,['f'],the longitudinal distribution function,['end']
299,the resonator voltage V f,['V'],the resonator voltage,['end']


In [64]:
# drop unecessary "[]" inside names
df["names"] = df["names"].apply(lambda name: name.strip("[]'"))
df['vars'] = df['vars'].apply(lambda x: x.strip("[]'"))
df.drop(columns=["variable_position"],inplace=True)

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  301 non-null    object
 1   vars      301 non-null    object
 2   names     301 non-null    object
dtypes: object(3)
memory usage: 7.2+ KB


In [66]:
df

,sentence,vars,names
0,The loading term becomes relevant at a time τ,τ,a time
1,we get easily for the rupture force f ∗,f,the rupture force
2,Green’s theorem can be used to show that the v...,d3xG(x),the volume
3,at a ﬁxed point x,x,point
4,it will be quite small (|K| is large) around t...,x,the point
...,...,...,...
296,we are inspired by the recent advances in the ...,(information),quantum
297,"then the correlation function C(A, B)","C(A, B)",the correlation function
298,Next is to utilize the perturbation expansion ...,f,the longitudinal distribution function
299,the resonator voltage V f,V,the resonator voltage


## Tokenization :

In [136]:
from spacy.tokenizer import Tokenizer
import re

def my_tokenizer(text,added_tokens) :

  # defining function pattern
  func_name = r"α-ωA-Za-zΑ-Ω0-9ℰℓℒℳøℂℕℙℚℝℤΓΔΛΞΠΣΦΨΩÅℏ∞∘∂∫∮∯∇αβγ∅"
  func_var = r"A-Za-zα-ωΑ-ΩℰℓℒℳøℂℕℙℚℝℤΓΔΛΞΠΣΦΨΩÅℏ∞∘∂∫∮∯∇αβγ∅"
  func_pattern = fr"(.|)([{func_name}]{{1,3}}(′|.|)\([{func_var}](,\s*[{func_var}])*\))"

  # getting math functions
  matches = re.findall(func_pattern, text)

  # getting only second-group matches (the functions)
  functions = [match[1] for match in matches]
  func_saver = iter(functions.copy())



  # Replace math functions with temporary markers
  for func in functions:
    text = text.replace(func, "[FUNC]")
    # adding the function to the gloabl variable
    added_tokens.append(func)


  # Tokenize the rest of the text
  tokens = text.split()

  # Replace temporary markers with original functions
  new_tokens = [next(func_saver) if '[FUNC]' in token else token for token in tokens]

  return new_tokens

In [138]:
def create_spacy_tokenizer(nlp):
    def custom_tokenizer(text):
        global added_tokens
        tokens = my_tokenizer(text,added_tokens)
        return Doc(nlp.vocab, words=tokens)
    return custom_tokenizer

In [143]:
import spacy
from spacy.tokens import Doc

nlp = spacy.load("en_core_web_sm")
nlp.tokenizer = create_spacy_tokenizer(nlp)

In [144]:
def tokenize(text) :
  tokens= []
  doc = nlp(text)

  # Iterate over the tokens in the processed doc
  for token in doc:
    tokens.append(str(token))
  return tokens

In [145]:
# this returns tokens inside the list
global added_tokens
added_tokens = []
df['tokenized_sentence'] = df['sentence'].apply(tokenize)

In [148]:
mathematical_symbols = [
    'α', 'β', 'γ', 'δ', 'ε', 'ζ', 'η', 'θ', 'ι', 'κ', 'λ', 'μ', 'ν', 'ξ', 'ο', 'π', 'ρ', 'σ', 'τ', 'υ', 'φ', 'χ', 'ψ', 'ω',
    'Α', 'Β', 'Γ', 'Δ', 'Ε', 'Ζ', 'Η', 'Θ', 'Ι', 'Κ', 'Λ', 'Μ', 'Ν', 'Ξ', 'Ο', 'Π', 'Ρ', 'Σ', 'Τ', 'Υ', 'Φ', 'Χ', 'Ψ', 'Ω',
    'ℰ', 'ℓ', 'ℒ', 'ℳ', 'ø', 'ℂ', 'ℕ', 'ℙ', 'ℚ', 'ℝ', 'ℤ','Γ', 'Δ', 'Λ', 'Ξ', 'Π', 'Φ', 'Ψ', 'Ω','Å', 'ℏ', '∞',
    '∂', '∮', '∯', '∇','∅','˜','µ','ǫ','ℋ', 'ℨ', 'ℛ']

mathematical_operations = ['∫', '∑', '∏', '√', '+', '-', '*', '/', '=', '^', '%','∩', '∪', '⊂', '⊆', '∈', '∉','∘' , "≡","<",">","↔","|"]

added_tokens = added_tokens + mathematical_symbols +  mathematical_operations

In [149]:
df.head()

,sentence,vars,names,tokenized_sentence
0,The loading term becomes relevant at a time τ,τ,a time,"[The, loading, term, becomes, relevant, at, a,..."
1,we get easily for the rupture force f ∗,f,the rupture force,"[we, get, easily, for, the, rupture, force, f, ∗]"
2,Green’s theorem can be used to show that the v...,d3xG(x),the volume,"[Green’s, theorem, can, be, used, to, show, th..."
3,at a ﬁxed point x,x,point,"[at, a, ﬁxed, point, x]"
4,it will be quite small (|K| is large) around t...,x,the point,"[it, will, be, quite, small, (|K|, is, large),..."


## Labelling :

In [150]:
sentence = ["The","quick","brown","fox" ,"jumps" ,"over","the","lazy","dog"]
word = "The quick brown fox"
word_tokened = word.split()
start = sentence.index(word_tokened[0])
end = sentence.index(word_tokened[-1])

for i in range(start,end+1) :
  if i == start :
    sentence[i] = "B-NAME"
  else :
    sentence[i] = "I-NAME"

In [151]:
def labeling(dataframe):
  labz = []
  for var , name , tokenized_sents in zip(dataframe["vars"],dataframe["names"],dataframe["tokenized_sentence"]) :
    # initilaising everything as oustide of entity
    labels = ['O']*len(tokenized_sents)
    # anonattating variable name
    if var in tokenized_sents :
      idx = tokenized_sents.index(var)
      labels[idx] = "B-VAR"

    tokenized_name = name.split()
    # dealing with names :
    if len(tokenized_name) > 1 :
      if tokenized_name[0] in  tokenized_sents :
        # getting name start index
        start_idx = tokenized_sents.index(tokenized_name[0])
        # getting name end index
        end_idx = tokenized_sents.index(tokenized_name[-1])

        # looping over the labels :
        for i in range(start_idx,end_idx+1) :
          if i == start_idx :
            labels[i] = "B-NAME"
          else :
            labels[i] = "I-NAME"
    elif len(tokenized_name) == 1 :
      if tokenized_name in tokenized_sents :
        idx = tokenized_sents.index(tokenized_name)
        labels[i] = "B-NAME"
    labz.append(labels)
  return labz

In [152]:
df["labels"] = labeling(df)
df

,sentence,vars,names,tokenized_sentence,labels
0,The loading term becomes relevant at a time τ,τ,a time,"[The, loading, term, becomes, relevant, at, a,...","[O, O, O, O, O, O, B-NAME, I-NAME, B-VAR]"
1,we get easily for the rupture force f ∗,f,the rupture force,"[we, get, easily, for, the, rupture, force, f, ∗]","[O, O, O, O, B-NAME, I-NAME, I-NAME, B-VAR, O]"
2,Green’s theorem can be used to show that the v...,d3xG(x),the volume,"[Green’s, theorem, can, be, used, to, show, th...","[O, O, O, O, O, O, O, O, B-NAME, I-NAME, O, O,..."
3,at a ﬁxed point x,x,point,"[at, a, ﬁxed, point, x]","[O, O, O, O, B-VAR]"
4,it will be quite small (|K| is large) around t...,x,the point,"[it, will, be, quite, small, (|K|, is, large),...","[O, O, O, O, O, O, O, O, O, B-NAME, I-NAME, B-..."
...,...,...,...,...,...
296,we are inspired by the recent advances in the ...,(information),quantum,"[we, are, inspired, by, the, recent, advances,...","[O, O, O, O, O, O, O, O, O, O, O, O, B-VAR, O]"
297,"then the correlation function C(A, B)","C(A, B)",the correlation function,"[then, the, correlation, function, C(A, B)]","[O, B-NAME, I-NAME, I-NAME, B-VAR]"
298,Next is to utilize the perturbation expansion ...,f,the longitudinal distribution function,"[Next, is, to, utilize, the, perturbation, exp...","[O, O, O, O, B-NAME, I-NAME, I-NAME, I-NAME, I..."
299,the resonator voltage V f,V,the resonator voltage,"[the, resonator, voltage, V, f]","[B-NAME, I-NAME, I-NAME, B-VAR, O]"


In [153]:
df["labels"][1]

['O', 'O', 'O', 'O', 'B-NAME', 'I-NAME', 'I-NAME', 'B-VAR', 'O']

In [157]:
def vocab_stats(model,tokenizer) :
  print(f"Tokenizer vocabulary size  : {len(tokenizer)}")
  print(f"Model embedding size : {model.get_input_embeddings().weight.shape[0]}")
  print("-"*50)

In [ ]:
# defining labels
label2id = {'O': 0, 'B-VAR': 1, 'B-NAME': 2, 'I-NAME': 3}
id2label = {v: k for k, v in label2id.items()}
num_labels = len(set(label for labels in df['labels'] for label in labels))

In [162]:
from transformers import AutoTokenizer,AutoModelForTokenClassification,DistilBertTokenizer,DistilBertForTokenClassification

# defining the tokenizer
my_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# vocab stats before adding the tokens
vocab_stats(model,my_tokenizer)

num_added_tokens = my_tokenizer.add_tokens(added_tokens)
print("number of tokens added :" ,num_added_tokens)

# vocab stats after adding the tokens
vocab_stats(model,my_tokenizer)

Tokenizer vocabulary size  : 30522
Model embedding size : 30589
--------------------------------------------------
number of tokens added : 67
Tokenizer vocabulary size  : 30589
Model embedding size : 30589
--------------------------------------------------


In [163]:
# defining the model + updating embedding table
model = DistilBertForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels=4, id2label=id2label, label2id=label2id)
model.resize_token_embeddings(len(my_tokenizer))

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(30589, 768)

In [164]:
# saving tokenizer
my_tokenizer.save_pretrained('./tokenizah')

('./tokenizah/tokenizer_config.json',
 './tokenizah/special_tokens_map.json',
 './tokenizah/vocab.txt',
 './tokenizah/added_tokens.json')

In [165]:
# checking non-added tokens
vocab = my_tokenizer.get_vocab()
for token in new_tokens:
    if token not in vocab:
        print(f"Token '{token}' is NOT in the vocabulary")

Token 'C(A, B)' is NOT in the vocabulary
Token 'V (t)' is NOT in the vocabulary
Token 'k(t)' is NOT in the vocabulary
Token 'S(ζ)' is NOT in the vocabulary
Token 'W(R)' is NOT in the vocabulary
Token '2(k)' is NOT in the vocabulary
Token 'T(R)' is NOT in the vocabulary
Token 'ng (x, y, z, t)' is NOT in the vocabulary
Token 'χ (z)' is NOT in the vocabulary
Token 'T (p, x)' is NOT in the vocabulary
Token 'R(Å)' is NOT in the vocabulary
Token 'U(x)' is NOT in the vocabulary
Token 'G(z)' is NOT in the vocabulary
Token '0(x)' is NOT in the vocabulary
Token 's(r)' is NOT in the vocabulary
Token 'f(R)' is NOT in the vocabulary
Token 'Z(k)' is NOT in the vocabulary
Token 'R(ξ, t)' is NOT in the vocabulary
Token '0(t)' is NOT in the vocabulary
Token '3xG(x)' is NOT in the vocabulary
Token 'mc(r)' is NOT in the vocabulary
Token 'i(z)' is NOT in the vocabulary
Token 'p(x, t)' is NOT in the vocabulary
Token 'M(t)' is NOT in the vocabulary
Token '1(ρ)' is NOT in the vocabulary
Token '12 (R)' is NOT

In [167]:
updated_tokenizer = DistilBertTokenizer.from_pretrained('./tokenizah')

id = updated_tokenizer.convert_tokens_to_ids("f(x)")
id

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


30549

In [23]:
pip install datasets

In [24]:
# Convert DataFrame to Dataset
from datasets import Dataset
dataset = Dataset.from_pandas(df)

In [25]:
dataset

Dataset({
    features: ['sentence', 'vars', 'names', 'tokenized_sentence', 'labels'],
    num_rows: 301
})

In [26]:
def DistillBERT_mapper(example):
  label_ids=[]
  input_ids = []
  attention_mask = []
  # Use the tokenized sentence directly
  labels = [sublist for sublist in example['labels']]
  sentences = [sent for sent in example["tokenized_sentence"]]

  for label in labels :

    label_ids.append([label2id[lab] for lab in label])



  # taking care of input_ids
  for sent in sentences :

    sent.insert(0, "[CLS]")
    sent.insert(len(sent), "[SEP]")
    my_input_ids = updated_tokenizer.convert_tokens_to_ids(sent)
    my_input_ids = my_input_ids[:128]  # Truncate if necessary
    my_input_ids.extend([0] * (128 - len(my_input_ids)))  # Pad if necessary
    input_ids.append(my_input_ids)



    my_attention_mask = [1] * len(my_input_ids)
    my_attention_mask = my_attention_mask[:128]  # Truncate if necessary
    my_attention_mask.extend([0] * (128 - len(my_attention_mask)))  # Pad if necessary
    attention_mask.append(my_attention_mask)

    #input_ids.append([0] * (128 - len(my_attention_mask)))


  # taking car of attention masks


  for label in label_ids :
    label_pad = 128 - len(label)
    #print(label_pad)
    label.extend([-100] * label_pad)

  return {
          'input_ids': input_ids,
          'attention_mask': attention_mask,
          'labels_ids': label_ids }

In [27]:
# Example input
example = {
    "tokenized_sentence": [["f(x)","is","a","function"]],
    "labels": [dataset[294]["labels"]]
}

# Call the mapper function
mapped_output = DistillBERT_mapper(example)

# Print the tokens and their IDs
print("Input IDs:", len(mapped_output['input_ids'][0]))
print("Attention Mask:", len(mapped_output['attention_mask'][0]))
print("Label IDs:", len(mapped_output['labels_ids'][0]))

Input IDs: 128
Attention Mask: 128
Label IDs: 128


In [28]:
 #tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

# Convert input IDs to tokens
tokens = updated_tokenizer.convert_ids_to_tokens(mapped_output['input_ids'][0])
print("Tokens:", tokens)

Tokens: ['[CLS]', 'f(x)', 'is', 'a', 'function', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]

In [29]:
# Apply the alignment function
encoded_dataset = dataset.map(DistillBERT_mapper,batched=True)

Map:   0%|          | 0/301 [00:00<?, ? examples/s]

In [30]:
encoded_dataset

Dataset({
    features: ['sentence', 'vars', 'names', 'tokenized_sentence', 'labels', 'input_ids', 'attention_mask', 'labels_ids'],
    num_rows: 301
})

In [31]:
# confirming the results
print(encoded_dataset[0])
ids = [0, 0, 0, 0, 0, 0, 2, 3, 1]
print([id2label[id] for id in ids])

{'sentence': 'The loading term becomes relevant at a time τ ', 'vars': 'τ', 'names': 'a time', 'tokenized_sentence': ['[CLS]', 'The', 'loading', 'term', 'becomes', 'relevant', 'at', 'a', 'time', 'τ', '[SEP]'], 'labels': ['O', 'O', 'O', 'O', 'O', 'O', 'B-NAME', 'I-NAME', 'B-VAR'], 'input_ids': [101, 100, 10578, 2744, 4150, 7882, 2012, 1037, 2051, 1174, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [32]:
# Columns to remove
columns_to_remove = ['sentence', 'vars', 'names', 'labels']

# Remove the unnecessary columns
dataset = encoded_dataset.remove_columns(columns_to_remove)

# Display the updated dataset to verify
print(dataset)

Dataset({
    features: ['tokenized_sentence', 'input_ids', 'attention_mask', 'labels_ids'],
    num_rows: 301
})


In [33]:
from sklearn.model_selection import train_test_split
from datasets import DatasetDict

split = dataset.train_test_split(test_size=0.2, seed=42)

# Creating DatasetDict
dataset_dict = DatasetDict({
    'train': split['train'],
    'validation': split['test']
})

In [34]:
new_column_names = {'tokenized_sentence': 'sentence', 'labels_ids': 'labels'}
dataset_dict = dataset_dict.rename_columns(new_column_names)

In [35]:
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['sentence', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 240
    })
    validation: Dataset({
        features: ['sentence', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 61
    })
})


## Training

In [36]:
!pip install evaluate
!pip install seqeval

In [37]:
label_list = list(label2id.keys())

In [38]:
label_list

['O', 'B-VAR', 'B-NAME', 'I-NAME']

In [39]:
import numpy as np
from seqeval.metrics import classification_report




def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)


    global true_predictions
    true_predictions  = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    global true_labels
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)


    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [58]:
from transformers import Trainer, TrainingArguments ,EarlyStoppingCallback
import numpy as np
from datasets import load_metric
import torch
import torch.nn as nn

import evaluate
seqeval = evaluate.load("seqeval")


# Adding early stopping :
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=5,  # Number of evaluation steps with no improvement before stopping
    early_stopping_threshold=0.01,  # Minimum change to qualify as an improvement
)


training_args = TrainingArguments(
    output_dir="./my-model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    logging_steps=100,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["validation"],
    tokenizer=updated_tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback]
)

# Start training
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.603486,0.687023,0.825688,0.750000,0.850769
2,No log,0.686739,0.699187,0.788991,0.741379,0.844615
3,No log,0.664645,0.734848,0.889908,0.804979,0.858462
4,No log,0.712287,0.701493,0.862385,0.773663,0.853846
5,No log,0.758468,0.723077,0.862385,0.786611,0.849231
6,No log,0.787544,0.739130,0.779817,0.758929,0.858462


TrainOutput(global_step=90, training_loss=0.06327240202162, metrics={'train_runtime': 19.4803, 'train_samples_per_second': 246.402, 'train_steps_per_second': 15.4, 'total_flos': 47036816916480.0, 'train_loss': 0.06327240202162, 'epoch': 6.0})

In [53]:
trainer.label_names

['labels']

In [59]:
from seqeval.metrics import classification_report

global true_predictions
global true_labels


report = classification_report(true_predictions, true_labels, output_dict=True)
df_report = pd.DataFrame(report)
df_report

,NAME,VAR,micro avg,macro avg,weighted avg
precision,0.666667,0.868852,0.779817,0.767760,0.765122
recall,0.542373,0.946429,0.739130,0.744401,0.739130
f1-score,0.598131,0.905983,0.758929,0.752057,0.748041
support,59.000000,56.000000,115.000000,115.000000,115.000000


In [42]:
trainer.state.log_history

[{'eval_loss': 0.9457947611808777,
  'eval_precision': 0.0,
  'eval_recall': 0.0,
  'eval_f1': 0.0,
  'eval_accuracy': 0.6015384615384616,
  'eval_runtime': 0.1554,
  'eval_samples_per_second': 392.532,
  'eval_steps_per_second': 25.74,
  'epoch': 1.0,
  'step': 15},
 {'eval_loss': 0.7588326334953308,
  'eval_precision': 0.2905982905982906,
  'eval_recall': 0.3119266055045872,
  'eval_f1': 0.3008849557522124,
  'eval_accuracy': 0.7076923076923077,
  'eval_runtime': 0.1512,
  'eval_samples_per_second': 403.344,
  'eval_steps_per_second': 26.449,
  'epoch': 2.0,
  'step': 30},
 {'eval_loss': 0.5955129861831665,
  'eval_precision': 0.5140845070422535,
  'eval_recall': 0.6697247706422018,
  'eval_f1': 0.5816733067729083,
  'eval_accuracy': 0.7738461538461539,
  'eval_runtime': 0.1504,
  'eval_samples_per_second': 405.6,
  'eval_steps_per_second': 26.597,
  'epoch': 3.0,
  'step': 45},
 {'eval_loss': 0.5857827067375183,
  'eval_precision': 0.5857142857142857,
  'eval_recall': 0.752293577981

In [43]:
for log in trainer.state.log_history :
  print(log)

{'eval_loss': 0.9457947611808777, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_accuracy': 0.6015384615384616, 'eval_runtime': 0.1554, 'eval_samples_per_second': 392.532, 'eval_steps_per_second': 25.74, 'epoch': 1.0, 'step': 15}
{'eval_loss': 0.7588326334953308, 'eval_precision': 0.2905982905982906, 'eval_recall': 0.3119266055045872, 'eval_f1': 0.3008849557522124, 'eval_accuracy': 0.7076923076923077, 'eval_runtime': 0.1512, 'eval_samples_per_second': 403.344, 'eval_steps_per_second': 26.449, 'epoch': 2.0, 'step': 30}
{'eval_loss': 0.5955129861831665, 'eval_precision': 0.5140845070422535, 'eval_recall': 0.6697247706422018, 'eval_f1': 0.5816733067729083, 'eval_accuracy': 0.7738461538461539, 'eval_runtime': 0.1504, 'eval_samples_per_second': 405.6, 'eval_steps_per_second': 26.597, 'epoch': 3.0, 'step': 45}
{'eval_loss': 0.5857827067375183, 'eval_precision': 0.5857142857142857, 'eval_recall': 0.7522935779816514, 'eval_f1': 0.6586345381526104, 'eval_accuracy': 0.795384615

In [44]:
import matplotlib.pyplot as plt
train_losses = [log['eval_loss'] for log in trainer.state.log_history[:-1]]
train_losses = [log['eval_loss'] for log in trainer.state.log_history[:-1]]

epochs = range(1, len(train_losses) + 1)

# Plot training loss
plt.plot(epochs, train_losses, label='Evaluation Loss')

# Extract other metrics as needed (e.g., validation loss, accuracy)
# ...

# Customize plot
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Evaluation Loss')
plt.legend()
plt.grid(True)
plt.show()

KeyError: 'eval_loss'

In [ ]:
import matplotlib.pyplot as plt

# Logs you provided
log_history = [
    # Your log data goes here
]

# Extracting losses and epochs
epochs = []
train_loss = []
val_loss = []

for log in log_history:
    if 'loss' in log and 'epoch' in log:
        epochs.append(log['epoch'])
        train_loss.append(log['loss'])
    if 'eval_loss' in log and 'epoch' in log:
        val_loss.append(log['eval_loss'])

# Plotting
plt.figure(figsize=(10, 5))
plt.plot(epochs[:len(train_loss)], train_loss, label='Training Loss')
plt.plot(epochs[len(train_loss):], val_loss, label='Validation Loss', color='red')

plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss per Epoch')
plt.legend()
plt.grid(True)


In [62]:
from transformers import pipeline
text = "n represents for the power"
updated_tokenizer = DistilBertTokenizer.from_pretrained('./tokenizah')


classifier = pipeline("ner",model="taissirboukrouba/my-model", device=0)
classifier(text)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[{'entity': 'I-NAME',
  'score': 0.73589855,
  'index': 4,
  'word': 'the',
  'start': 17,
  'end': 20},
 {'entity': 'I-NAME',
  'score': 0.39467815,
  'index': 5,
  'word': 'power',
  'start': 21,
  'end': 26}]

In [ ]:
classifier.steps